In [1]:
# functions to calculation bio fluxes

In [56]:
#convert kg/m2/s to kg/3hours
# TODO: make it more general, take input temporal resolution in hours???
def unit_convert_tmp(dr, res_lat, res_lon):
    # use grid area function
    for ilat, lat in enumerate(dr.lat.values):
        area = 1e6 * area_latlon(lat1 = lat, lat2 = lat + res_lat,
                                 lon1 = 10, lon2 = 10 + res_lon) # m^2, longitude doesn't matter
        dr[:,ilat,:] = dr[:,ilat,:] * area * (3600*3) # kg/3hours
    return dr

In [ ]:
def monthly_bio(fname): 
    ds_original = xr.open_dataset(fname)

    dr_unit = ds_original['CO2'].copy()
    dr_output = unit_convert_tmp(dr_unit, 1., 1.25) # unit converted to kg/3hours per grid cell

    # compute regional total
    data = ds_original['CO2'].copy()
    # create masks for this resolution
    dr_mask = data[0,:,:]
    # create masks for input file
    mask = create_masks(input_file=dr_mask, 
                        input_file_type='dataarray', 
                        var_name='', 
                        mask_type='giorgi', 
                        output_format='xarray', 
                        output_path='/geos/u73/msadiq/GEOS-Chem/MASKS/nc/giorgi/', 
                        figure_flag = False, 
                        figure_path='/home/msadiq/Desktop/co2/data/emission/MASKS/figures/')
    # split a mask into separate dataarrays (one dataset), with 1s and 0s
    ds_mask_splits = split_mask(input_mask = mask)

    # sum up monthly from 3 hourly data
    dr_monthly = ds_original['CO2'][:12,:,:].copy()
    dr_monthly[:,:,:] = 0.
    # fix time dimension
    months = np.arange('2016-01', '2017-02', dtype='datetime64[M]')
    dr_monthly['time'] = months[:12]

    for i in range(12):
        before = dr_output.sel(time = slice(months[i], months[i+1]))
        dr_monthly[i,:,:] = before.sum(axis=0) # unit: kgCO2/month
    
    return dr_monthly


In [57]:
def regional_monthly_bio(fname): 
    ds_original = xr.open_dataset(fname)

    dr_unit = ds_original['CO2'].copy()
    dr_output = unit_convert_tmp(dr_unit, 1., 1.25) # unit converted to kg/3hours per grid cell

    # compute regional total
    data = ds_original['CO2'].copy()
    # create masks for this resolution
    dr_mask = data[0,:,:]
    # create masks for input file
    mask = create_masks(input_file=dr_mask, 
                        input_file_type='dataarray', 
                        var_name='', 
                        mask_type='giorgi', 
                        output_format='xarray', 
                        output_path='/geos/u73/msadiq/GEOS-Chem/MASKS/nc/giorgi/', 
                        figure_flag = False, 
                        figure_path='/home/msadiq/Desktop/co2/data/emission/MASKS/figures/')
    # split a mask into separate dataarrays (one dataset), with 1s and 0s
    ds_mask_splits = split_mask(input_mask = mask)

    # sum up monthly from 3 hourly data
    dr_monthly = ds_original['CO2'][:12,:,:].copy()
    dr_monthly[:,:,:] = 0.
    # fix time dimension
    months = np.arange('2016-01', '2017-02', dtype='datetime64[M]')
    dr_monthly['time'] = months[:12]

    for i in range(12):
        before = dr_output.sel(time = slice(months[i], months[i+1]))
        dr_monthly[i,:,:] = before.sum(axis=0) # unit: kgCO2/month
    
    # monthly regional total flux
    nx = 22
    monthly_bio_flux = np.zeros((12,nx))

    for i in range(12):
        for j in range(nx):
            mask_name = 'MASK' + str(j+1)
            tmp = dr_monthly[i,:,:] * ds_mask_splits[mask_name]
            monthly_bio_flux[i,j] = tmp.sum().values*1e-12*12/44 # unit: PgC/month

    print('Biospheric annual flux (PgC/year): ', np.sum(monthly_bio_flux))
    return monthly_bio_flux
